In [1]:
'''
Author: qianyu
Date: 2026-01-20 15:48:52
LastEditTime: 2026-01-20 16:41:06
'''
import os
import yt_dlp

def get_download_options(url, cookies_file):
    """
    获取YouTube视频的所有可下载格式列表
    :param url: YouTube视频链接
    :param cookies_file: cookies文件路径
    :return: 视频信息字典（包含所有格式），或None
    """
    try:
        ydl_opts = {
            'quiet': True,          # 静默模式，只获取信息不下载
            'no_warnings': True,
            'cookiefile': cookies_file,
            'skip_download': True,  # 跳过实际下载，仅获取信息
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # 提取视频信息（包含所有可下载格式）
            video_info = ydl.extract_info(url, download=False)
            return video_info

    except Exception as e:
        print(f"获取下载选项失败：{str(e)}")
        return None

def print_download_options(video_info):
    """
    格式化打印所有可下载的视频/音频选项
    :param video_info: 视频信息字典
    """
    if not video_info or 'formats' not in video_info:
        print("未获取到可下载的格式信息")
        return

    print("\n" + "="*80)
    print(f"视频标题：{video_info.get('title', '未知')}")
    print(f"视频ID：{video_info.get('id', '未知')}")
    print("="*80)
    print("可下载格式列表（格式ID | 类型 | 分辨率/码率 | 扩展名 | 大小）：")
    print("-"*80)

    # 遍历所有格式，筛选并格式化输出
    for fmt in video_info['formats']:
        # 跳过不需要的格式（比如m3u8流媒体）
        if fmt.get('vcodec') == 'none' and fmt.get('acodec') == 'none':
            continue
        
        # 基础信息
        fmt_id = fmt.get('format_id', '未知')
        ext = fmt.get('ext', '未知')
        filesize = fmt.get('filesize') or fmt.get('filesize_approx')
        filesize_str = f"{filesize/1024/1024:.2f} MB" if filesize else "未知"
        
        # 视频/音频类型判断
        if fmt.get('vcodec') != 'none' and fmt.get('acodec') != 'none':
            type_str = "视频+音频"
            resolution = fmt.get('resolution', '未知') or f"{fmt.get('width', '?')}x{fmt.get('height', '?')}"
        elif fmt.get('vcodec') != 'none':
            type_str = "仅视频"
            resolution = fmt.get('resolution', '未知') or f"{fmt.get('width', '?')}x{fmt.get('height', '?')}"
        else:
            type_str = "仅音频"
            resolution = f"{fmt.get('abr', '?')} kbps"
        
        # 打印一行
        print(f"{fmt_id:>6} | {type_str:<8} | {resolution:<12} | {ext:<6} | {filesize_str}")

    print("="*80)

def download_youtube_video(url, cookies_file, format_id=None):
    """
    使用yt-dlp下载YouTube视频，带cookies认证，可指定格式
    :param url: YouTube视频链接
    :param cookies_file: cookies文件路径
    :param format_id: 指定下载的格式ID（None则下载默认最佳格式）
    """
    try:
        # 创建输出目录（如果不存在）
        output_dir = 'data/raw_videos'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"创建输出目录：{output_dir}")

        # 下载配置
        ydl_opts = {
            'outtmpl': os.path.join(output_dir, '%(title)s_%(id)s.%(ext)s'),  # 输出路径和文件名格式
            'quiet': False,  # 显示下载进度
            'no_warnings': True,  # 屏蔽警告
            'cookiefile': cookies_file,  # 使用cookies文件
        }

        # 如果指定了格式ID，添加格式筛选
        if format_id:
            ydl_opts['format'] = format_id
            print(f"指定下载格式ID：{format_id}")

        # 执行下载
        print(f"\n正在下载视频：{url}")
        print(f"使用cookies文件：{cookies_file}")
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
            
        print("\n下载完成！文件保存至：", output_dir)
        return True
        
    except Exception as e:
        print(f"\n下载失败：{str(e)}")
        return False

if __name__ == "__main__":
    # 测试视频URL
    test_url = "https://www.youtube.com/shorts/U9vZjYmt81c"
    
    # cookies文件路径
    cookies_path = os.path.join("config", "cookies.txt")
    
    # 检查cookies文件是否存在
    if not os.path.exists(cookies_path):
        print(f"错误：cookies文件不存在 → {cookies_path}")
        print("请确保您已经在config目录下放置了cookies.txt文件")
    else:
        # 1. 获取并打印所有可下载选项
        video_info = get_download_options(test_url, cookies_path)
        if video_info:
            print_download_options(video_info)
            
            # 2. 可选：让用户输入格式ID进行下载（注释掉则自动下载最佳格式）
            user_choice = input("\n请输入要下载的格式ID（直接回车下载默认最佳格式）：").strip()
            format_id = user_choice if user_choice else None
            
            # 3. 执行下载
            download_youtube_video(test_url, cookies_path, format_id)




视频标题：knowledge Distillation
视频ID：U9vZjYmt81c
可下载格式列表（格式ID | 类型 | 分辨率/码率 | 扩展名 | 大小）：
--------------------------------------------------------------------------------
  91-0 | 视频+音频    | 144x256      | mp4    | 未知
  91-1 | 视频+音频    | 144x256      | mp4    | 未知
  91-2 | 视频+音频    | 144x256      | mp4    | 未知
  91-3 | 视频+音频    | 144x256      | mp4    | 未知
  91-4 | 视频+音频    | 144x256      | mp4    | 未知
  91-5 | 视频+音频    | 144x256      | mp4    | 未知
  91-6 | 视频+音频    | 144x256      | mp4    | 未知
  91-7 | 视频+音频    | 144x256      | mp4    | 未知
  91-8 | 视频+音频    | 144x256      | mp4    | 未知
  91-9 | 视频+音频    | 144x256      | mp4    | 未知
 91-10 | 视频+音频    | 144x256      | mp4    | 未知
 91-11 | 视频+音频    | 144x256      | mp4    | 未知
 91-12 | 视频+音频    | 144x256      | mp4    | 未知
 91-13 | 视频+音频    | 144x256      | mp4    | 未知
  92-0 | 视频+音频    | 240x426      | mp4    | 未知
  92-1 | 视频+音频    | 240x426      | mp4    | 未知
  92-2 | 视频+音频    | 240x426      | mp4    | 未知
  92-3 | 视频+音频    | 240x426      |

KeyboardInterrupt: 